In [6]:
print('hello')

hello


In [3]:
# !pip install pymssql

In [3]:
key_vault_name = 'nc6285-kv-y2zgab2u5ycoy'

import pandas as pd
import pyodbc
import os
import pymssql

from azure.keyvault.secrets import SecretClient  
from azure.identity import DefaultAzureCredential 

def get_secrets_from_kv(kv_name, secret_name):
    key_vault_name = kv_name  # Set the name of the Azure Key Vault  
    credential = DefaultAzureCredential()
    secret_client = SecretClient(vault_url=f"https://{key_vault_name}.vault.azure.net/", credential=credential)  # Create a secret client object using the credential and Key Vault name  
    return(secret_client.get_secret(secret_name).value) # Retrieve the secret value  

server = get_secrets_from_kv(key_vault_name,"SQLDB-SERVER")
database = get_secrets_from_kv(key_vault_name,"SQLDB-DATABASE")
username = get_secrets_from_kv(key_vault_name,"SQLDB-USERNAME")
password = get_secrets_from_kv(key_vault_name,"SQLDB-PASSWORD")

# connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
# print(connection_string)
# conn = pyodbc.connect(connection_string)

conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()


from azure.storage.filedatalake import (
    DataLakeServiceClient
)

account_name = get_secrets_from_kv(key_vault_name, "ADLS-ACCOUNT-NAME")
account_key = get_secrets_from_kv(key_vault_name, "ADLS-ACCOUNT-KEY")

account_url = f"https://{account_name}.dfs.core.windows.net"

service_client = DataLakeServiceClient(account_url, credential=account_key,api_version='2023-01-03') 


file_system_client_name = "data"
directory = 'clientdata' 

file_system_client = service_client.get_file_system_client(file_system_client_name)  
directory_name = directory

cursor = conn.cursor()

In [10]:
cursor.execute('DROP TABLE IF EXISTS Clients1')
conn.commit()

create_client_sql = """CREATE TABLE Clients1 (
                ClientId int NOT NULL PRIMARY KEY,
                Client varchar(255),
                Email varchar(255),
                Occupation varchar(255),
                MaritalStatus varchar(255),
                Dependents int
            );"""
cursor.execute(create_client_sql)
conn.commit()

# Read the CSV file into a Pandas DataFrame
file_path = directory + '/Clients.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

# df = pd.read_csv('../Data/Clients.csv')
# df.head()

for index, item in df.iterrows():
    #cursor.execute(f"INSERT INTO Clients1 (ClientId,Client, Email, Occupation, MaritalStatus, Dependents) VALUES (?,?, ?, ?, ?, ?)", (item.ClientId, item.Client, item.Email, item.Occupation, item.MaritalStatus, item.Dependents))
    cursor.execute(f"INSERT INTO Clients1 (ClientId,Client, Email, Occupation, MaritalStatus, Dependents) VALUES (%s,%s, %s, %s, %s, %d)", (item.ClientId, item.Client, item.Email, item.Occupation, item.MaritalStatus, item.Dependents))
    
conn.commit()

In [4]:
from os import getenv
import pymssql

conn = pymssql.connect(server, user, password, "bycwa-sqldb")
cursor = conn.cursor()
# cursor.execute("""
# IF OBJECT_ID('persons', 'U') IS NOT NULL
#     DROP TABLE persons
# CREATE TABLE persons (
#     id INT NOT NULL,
#     name VARCHAR(100),
#     salesrep VARCHAR(100),
#     PRIMARY KEY(id)
# )
# """)
# cursor.executemany(
#     "INSERT INTO persons VALUES (%d, %s, %s)",
#     [(1, 'John Smith', 'John Doe'),
#      (2, 'Jane Doe', 'Joe Dog'),
#      (3, 'Mike T.', 'Sarah H.')])
# # you must call commit() to persist your data if you don't set autocommit to True
# conn.commit()

cursor.execute('SELECT * FROM clients')
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

conn.close()

(10001, 'Alexander Harrington', 'Alexander.Harrington@contoso.com', 'Architect', 'Married', 2)
(10002, 'Amelia Harper', 'Amelia.Harper@contoso.com', 'Engineer', 'Single', 0)
(10003, 'Arun Sharma', 'Arun.Sharma@contoso.com', 'Doctor', 'Married', 3)
(10004, 'Isabella Simmons', 'Isabella.Simmons@contoso.com', 'Engineer', 'Married', 0)
(10005, 'Karen Berg', 'Karen.Berg@contoso.com', 'Architect', 'Married', 2)
(10006, 'Victoria Kensington', 'Victoria.Kensington@contoso.com', 'Designer', 'Single', 1)


In [ ]:
key_vault_name = 'kv_to-be-replaced'

import pandas as pd
import pyodbc
import os

from azure.keyvault.secrets import SecretClient  
from azure.identity import DefaultAzureCredential 

def get_secrets_from_kv(kv_name, secret_name):
    key_vault_name = kv_name  # Set the name of the Azure Key Vault  
    credential = DefaultAzureCredential()
    secret_client = SecretClient(vault_url=f"https://{key_vault_name}.vault.azure.net/", credential=credential)  # Create a secret client object using the credential and Key Vault name  
    return(secret_client.get_secret(secret_name).value) # Retrieve the secret value  

server = get_secrets_from_kv(key_vault_name,"SQLDB-SERVER")
database = get_secrets_from_kv(key_vault_name,"SQLDB-DATABASE")
username = get_secrets_from_kv(key_vault_name,"SQLDB-USERNAME")
password = get_secrets_from_kv(key_vault_name,"SQLDB-PASSWORD")

connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
print(connection_string)
conn = pyodbc.connect(connection_string)


from azure.storage.filedatalake import (
    DataLakeServiceClient
)

account_name = get_secrets_from_kv(key_vault_name, "ADLS-ACCOUNT-NAME")
account_key = get_secrets_from_kv(key_vault_name, "ADLS-ACCOUNT-KEY")

account_url = f"https://{account_name}.dfs.core.windows.net"

service_client = DataLakeServiceClient(account_url, credential=account_key,api_version='2023-01-03') 


file_system_client_name = "data"
directory = 'clientdata' 

file_system_client = service_client.get_file_system_client(file_system_client_name)  
directory_name = directory

cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Clients')
conn.commit()

create_client_sql = """CREATE TABLE Clients (
                ClientId int NOT NULL PRIMARY KEY,
                Client varchar(255),
                Email varchar(255),
                Occupation varchar(255),
                MaritalStatus varchar(255),
                Dependents int
            );"""
cursor.execute(create_client_sql)
conn.commit()

# Read the CSV file into a Pandas DataFrame
file_path = directory + '/Clients.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

# df = pd.read_csv('../Data/Clients.csv')
# df.head()

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO Clients (ClientId,Client, Email, Occupation, MaritalStatus, Dependents) VALUES (?,?, ?, ?, ?, ?)", item.ClientId, item.Client, item.Email, item.Occupation, item.MaritalStatus, item.Dependents)
conn.commit()

# cursor.execute(f"select * from Clients")
# for row in cursor.fetchall():
#     print(row.ClientId, row.Client, row.Email)


cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS ClientInvestmentPortfolio')
conn.commit()

create_client_sql = """CREATE TABLE ClientInvestmentPortfolio (
                ClientId int,
                AssetDate date,
                AssetType varchar(255),
                Investment float,
                ROI float,
                RevenueWithoutStrategy float
            );"""

cursor.execute(create_client_sql)
conn.commit()

# df = pd.read_csv('../Data/ClientInvestmentPortfolio.csv')
# df.head()

file_path = directory + '/ClientInvestmentPortfolio.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO ClientInvestmentPortfolio (ClientId, AssetDate, AssetType, Investment, ROI, RevenueWithoutStrategy) VALUES (?,?, ?, ?, ?, ?)", item.ClientId, item.AssetDate, item.AssetType, item.Investment, item.ROI, item.RevenueWithoutStrategy)
    
conn.commit()

# cursor.execute(f"select * from ClientInvestmentPortfolio")
# for row in cursor.fetchall():
#     print(row.ClientId, row.AssetType, row.Investment)


from decimal import Decimal

cursor.execute('DROP TABLE IF EXISTS Assets')
conn.commit()

create_assets_sql = """CREATE TABLE Assets (
                ClientId int NOT NULL,
                AssetDate Date,
                Investment Decimal(18,2),
                ROI Decimal(18,2),
                Revenue Decimal(18,2),
                AssetType varchar(255)
            );"""

cursor.execute(create_assets_sql)
conn.commit()

# df = pd.read_csv('../Data/Assets.csv')
file_path = directory + '/Assets.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

df['AssetDate'] = pd.to_datetime(df['AssetDate'], format='%m/%d/%Y') #   %Y-%m-%d')
df['ClientId'] = df['ClientId'].astype(int)
df['Investment'] = df['Investment'].astype(float)
df['ROI'] = df['ROI'].astype(float)
df['Revenue'] = df['Revenue'].astype(float)


for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO Assets (ClientId,AssetDate, Investment, ROI, Revenue, AssetType) VALUES (?,?, ?, ?, ?, ?)", item.ClientId, item.AssetDate, item.Investment, item.ROI, item.Revenue, item.AssetType)
conn.commit()

# cursor.execute(f"select * from Assets")
# for row in cursor.fetchall():
#     print(row.Investment, row.ROI)

cursor.execute('DROP TABLE IF EXISTS InvestmentGoals')
conn.commit()

create_ig_sql = """CREATE TABLE InvestmentGoals (
                ClientId int NOT NULL,
                InvestmentGoal varchar(255)
            );"""

cursor.execute(create_ig_sql)
conn.commit()

# df = pd.read_csv('../Data/InvestmentGoals.csv')


file_path = directory + '/InvestmentGoals.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

df['ClientId'] = df['ClientId'].astype(int)

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO InvestmentGoals (ClientId,InvestmentGoal) VALUES (?,?)", item.ClientId, item.InvestmentGoal)
conn.commit()

# cursor.execute(f"select * from InvestmentGoals")
# for row in cursor.fetchall():
#     print(row.ClientId,row.InvestmentGoal)

cursor.execute('DROP TABLE IF EXISTS InvestmentGoalsDetails')
conn.commit()

create_ig_sql = """CREATE TABLE InvestmentGoalsDetails (
                ClientId int NOT NULL,
                InvestmentGoal nvarchar(255), 
                TargetAmount Decimal(18,2), 
                Contribution Decimal(18,2), 
            );"""

cursor.execute(create_ig_sql)
conn.commit()

# df = pd.read_csv('../Data/InvestmentGoalsDetails.csv')
file_path = directory + '/InvestmentGoalsDetails.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

df['ClientId'] = df['ClientId'].astype(int)

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO InvestmentGoalsDetails (ClientId,InvestmentGoal, TargetAmount, Contribution) VALUES (?,?,?,?)", item.ClientId, item.InvestmentGoal, item.TargetAmount, item.Contribution)
conn.commit()

# cursor.execute(f"select * from InvestmentGoalsDetails")
# for row in cursor.fetchall():
#     print(row.ClientId,row.InvestmentGoal)

cursor.execute('DROP TABLE IF EXISTS ClientSummaries')
conn.commit()

create_cs_sql = """CREATE TABLE ClientSummaries (
                ClientId int NOT NULL,
                ClientSummary nvarchar(255)
            );"""

cursor.execute(create_cs_sql)
conn.commit()

# df = pd.read_csv('../Data/ClientSummaries.csv')
file_path = directory + '/ClientSummaries.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

df['ClientId'] = df['ClientId'].astype(int)

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO ClientSummaries (ClientId,ClientSummary) VALUES (?,?)", item.ClientId, item.ClientSummary)
conn.commit()

# cursor.execute(f"select * from ClientSummaries")
# for row in cursor.fetchall():
#     print(row.ClientId,row.ClientSummary)

cursor.execute('DROP TABLE IF EXISTS Retirement')
conn.commit()

create_cs_sql = """CREATE TABLE Retirement (
                ClientId int NOT NULL,
                StatusDate Date,
                RetirementGoalProgress Decimal(18,2),
                EducationGoalProgress Decimal(18,2)
            );"""

cursor.execute(create_cs_sql)
conn.commit()

# df = pd.read_csv('../Data/Retirement.csv')
file_path = directory + '/Retirement.csv'
file_client = file_system_client.get_file_client(file_path)
csv_file = file_client.download_file()
df = pd.read_csv(csv_file, encoding='utf-8')

df['ClientId'] = df['ClientId'].astype(int)

for index, item in df.iterrows():
    cursor.execute(f"INSERT INTO Retirement (ClientId,StatusDate, RetirementGoalProgress, EducationGoalProgress) VALUES (?,?,?,?)", item.ClientId, item.StatusDate, item.RetirementGoalProgress, item.EducationGoalProgress)
conn.commit()

# cursor.execute(f"select * from Retirement")
# for row in cursor.fetchall():
#     print(row.ClientId,row.RetirementGoalProgress)


# to adjust dates in meetings table
cursor = conn.cursor()
sql_query = 'select 7 - datediff(day,getdate(), max(cast([StartTime] as date))) as n FROM ClientMeetings'
cursor.execute(sql_query)
for row in cursor.fetchall():
    ndays = row.n
    print(row.n)
    break

sql_query = f'UPDATE ClientMeetings SET StartTime = DATEADD (day, {ndays}, StartTime)'
cursor.execute(sql_query)
sql_query = f'UPDATE ClientMeetings SET EndTime = DATEADD (day, {ndays}, EndTime)'
cursor.execute(sql_query)
conn.commit()